# Example Usage for GeoCluster Package

In [4]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))


## Python Version
import sys
print("Python: {0}".format(sys.version))


## Install
from timeUtils import clock, elapsed
from ioUtils import saveJoblib
from geocluster import geoClusters
from geoUtils import convertMetersToLat, convertLatToMeters, convertMetersToLong, convertLongToMeters
from geoclusterUtils import genCenters, genCluster, genClusters, genTripsBetweenClusters

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.6.5 |Anaconda custom (x86_64)| (default, Apr 26 2018, 08:42:37) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2018-11-15 21:14:24.217071










































# Example GPS Data

In [5]:
genMax  = 75
distMax = 500
raw = genClusters(20, 250, latRange=[29.8, 30.2], lngRange=[49.8, 50.2], dist="gauss", maxrad=genMax)

In [6]:
def plotMeters(ax1, longMeters, latMeters):
    ax2 = ax1.twinx()
    ax2.plot(longMeters, latMeters, color='b', lw=0)
    ax3 = ax1.twiny()
    ax3.plot(longMeters, latMeters, color='b', lw=0)
    
def plotRawData(rawdata, color='cyan'):
    import seaborn as sns
    from matplotlib import pyplot as plt 
    fig, ax1 = plt.subplots()
    lat = rawdata[:,0]
    long = rawdata[:,1]
    ax1.scatter(long, lat, s=15, linewidth=0, color='cyan', alpha=1) #c=cluster_member_colors, alpha=1)
    return ax1

def clusterData(rawdata, distMax):
    %load_ext autoreload
    %autoreload
    gc = geoClusters(key="test", points=rawdata, distMax=distMax, debug=True)
    gc.findClusters(seedMin=2, debug=True)
    if True:
        print("Found {0} clusters using {1} cells and {2} counts".format(gc.getNClusters(), gc.getNCells(), gc.getNCounts()))
    return gc

def plotClusters(ax1, gc, color='red'):
    from seaborn import color_palette
    from matplotlib.patches import Circle, Wedge, Polygon
    from matplotlib.collections import PatchCollection
    
    clusters = gc.getClusters()
    coms = gc.getClusterCoMs()
    color_palette = color_palette('deep', 2)
    patches = []
    print("Plotting {0} clusters".format(len(clusters)))
    for cl, cluster in clusters.items():
        radius = cluster.getRadius()
        com    = cluster.getCoM()
        quant  = cluster.getQuantiles()
        
        radius = quant[-1]
        ax1.scatter(com[1], com[0], s=10, marker='x', linewidth=2, c='black', alpha=1)

        latDist = convertMetersToLat(radius)
        circle = Circle(xy=(com[1], com[0]), radius=latDist)
        patches.append(circle)

    p = PatchCollection(patches, facecolor='red', alpha=0.25)
    from numpy import array, linspace
    #p.set_array(linspace(0,1,len(pcols)))
    ax1.add_collection(p)

#latOff = lat - min(lat)
#latMeters = convertLatToMeters(latOff)
#lngOff = long - min(long)
#lngMeters = convertLongToMeters(lngOff, lat)

#plotMeters(ax1, latMeters, lngMeters)
gc  = clusterData(raw, distMax=distMax)
ax1 = plotRawData(raw)
ax1 = plotClusters(ax1, gc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Thu Nov 15, 2018 21:14:27 for Finding Geohash (BitLen=8) Values from 5000 Points
Current Time is Thu Nov 15, 2018 21:14:27 for Done with Finding Geohash (BitLen=8) Values from 5000 Points
Process [Done with Finding Geohash (BitLen=8) Values from 5000 Points] took 0 seconds.
Current Time is Thu Nov 15, 2018 21:14:27 for Finding Geohash (BitLen=8) Frequency Values from Geohash DataFrame
Current Time is Thu Nov 15, 2018 21:14:27 for Done with Finding Geohash (BitLen=8) Frequency Values from Geohash DataFrame
Process [Done with Finding Geohash (BitLen=8) Frequency Values from Geohash DataFrame] took 0 seconds.
Current Time is Thu Nov 15, 2018 21:14:27 for Finding Clusters with at least 2 counts
  --> Creating cluster cl0 with seed tj76jb3e and 14 counts
  --> Creating cluster cl1 with seed tj7dfh73 and 10 counts
  --> Creating cluster cl2 with seed tj77m2sf and 10 counts
  --> Creating c

## Generate Random Data From Clusters

In [36]:
%load_ext autoreload
%autoreload

from geoclusterUtils import genCenters, genCluster, genClusters, genTripsBetweenClusters
data = genTripsBetweenClusters(1000, gc, returnLoc=True, returnDF=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Selected 1000 randomized trips
Found Start/End for the 1000 randomized trips
Converting (1000, 2, 2) trips to a DataFrame


In [37]:
saveJoblib(data, "../network/trips.p")

  --> This file is 97.8kB.


In [143]:
x = genTripsBetweenClusters(100, gc, returnDF=True)

Selected 100 randomized trips
Found Start/End for the 100 randomized trips
Converting (100, 2, 2) trips to a DataFrame
